# Molecular Docking on AutoDock

### This notebook was adapted from the original Lab.06 / IBM3202 – Molecular Docking on AutoDock.
It forms part of a very good bioinformatics series which can be found [here](https://github.com/pb3lab/ibm3202).
This was released under and MIT license.

### Citation:
Engelberger F, Galaz-Davison P, Bravo G, Rivera M, Ramírez-Sarmiento CA (2021) Developing and Implementing Cloud-Based Tutorials that Combine Bioinformatics Software, Interactive Coding and Visualization Exercises for Distance Learning on Structural Bioinformatics. J Chem Educ 98(5): 1801-1807. doi: 10.1021/acs.jchemed.1c00022

<a rel="license" href="https://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons Licence" style="width=50" src="https://licensebuttons.net/l/by-nc-sa/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>



<a rel="license" href="https://opensource.org/licenses/MIT "><img alt="MIT Licence" src="https://img.shields.io/badge/License-MIT-yellow.svg" align="left"/></a>

<br></br>
**Authors**: Engelberger F, Galaz-Davison P, Bravo G, Rivera M, Ramírez-Sarmiento CA   
**Contributor**: Dr Antonia Mey (antonia.mey@ed.ac.uk) and Rohan Gorantla

**Questions:** Can we create a protein-ligand complex from a crystal structure and a smiles string?

**Objectives:**
  - Prepare a protein for docking with Autodock Vina
  - Prepare a series of ligands for Docking
  - Understand docking results by computing RMSDs and comparing scores against experimental results

**Jupyter cheat sheet**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;

## Google Colab setup

<div class="alert alert-warning">
<b>Attention:</b> Please only run the following cell if you are using Colab! This cell will download the data required for Section 4 of this tutorial.</div>

In [ ]:
!if [ -n "$COLAB_RELEASE_TAG" ]; then git clone https://github.com/CCPBioSim/CCP5_Simulation_of_BioMolecules; fi
import os
os.chdir(f"CCP5_Simulation_of_BioMolecules{os.sep}4_Docking")

## Table of Contents
[0.   Setup](#setup)   
[1.   Theoretical background](#theory)   
[2.   Protein prep](#protein)   
[3.   Ligand prep](#ligand)   
[4.   Docking](#docking)   

# 0. Setting up and downloading and installing the required software
<a id='setup'></a>

The following is for **Google Colab** only!

This tutorial has many dependencies for it to work. They are:
- **biopython** for manipulation of the PDB files
- **py3Dmol** to visualise the protein structure and set up the search grid.
- **miniconda**, a free minimal installer of **conda** for software package and environment management.
- **OpenBabel** for parameterization of our ligand(s).
- **MGLtools** are used to parameterize our target protein using Gasteiger charges.
- **pdb2pqr** for parameterization of our protein using the AMBER ff99 forcefield.
- **Autodock Vina** for the docking process

After several tests, the following installation instructions are the best way of setting up **Google Colab** for this laboratory session.

### 0.1 Setup conda colab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### 0.2 Check conda colab works

In [ ]:
import condacolab
condacolab.check()

### 0.3 Install py3D mol for visualisation and pdb2pqr for parametrisation

In [ ]:
!pip install py3Dmol
!pip install pdb2pqr

### 0.4 Creating and installing the rest of the environment

In [ ]:
# Step 1: Create the env.yml file
env_yaml = """
channels:
  - conda-forge

dependencies:
  - python >= 3.8
  - pip
  - tqdm
  - loguru
  - fsspec
  - s3fs
  - gcsfs
  - joblib

  # Scientific
  - pandas = 2.1
  - numpy
  - scipy

  # Chemistry
  - rdkit
  - datamol
  - openmm >=7.6.0
  - pdbfixer >=1.8
  - openmmforcefields
  - mdtraj
  - mdanalysis

  # Docking
  - openbabel
  - smina
  - qvina
  - fpocket

  # Viz
  - matplotlib
  - ipywidgets = 7.7.2
  - nglview
"""

# Write the environment configuration to a file
with open("environment.yml", "w") as file:
    file.write(env_yaml)


And now we can install the environment we have created using mamba. Please be patient! **The next two cells may take some time to run**.

In [ ]:
%%capture
!mamba env update -n base -f environment.yml

In [ ]:
%%capture
!mamba install -c conda-forge -c bioconda mgltools openbabel zlib ncurses --yes

In [ ]:
# Importing py3Dmol for safety
import py3Dmol

In [ ]:
# Checking that pdb2pqr was properly installed
!pdb2pqr30 -h | awk 'NR==1{if($1=="usage:") print "PDB2PQR succesfully installed"; else if($1!="usage:") print "Something went wrong. Please install again"}'

### 0.5 Autodock Vina
Finally, we will download the Autodock Vina program from the [AutoDock Vina GitHub](https://github.com/ccsb-scripps/AutoDock-Vina) and make an alias to use it during this session

In [ ]:
# Download and extract Autodock Vina from SCRIPPS
# Then, we set up an alias for vina to be treated as a native binary
%%bash
mkdir vina
cd vina
wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
chmod +x /content/vina/vina_1.2.5_linux_x86_64
wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_split_1.2.5_linux_x86_64
chmod +x /content/vina/vina_split_1.2.5_linux_x86_64

In [ ]:
%alias vina /content/vina/vina_1.2.5_linux_x86_64
%alias vina_split /content/vina/vina_1.2.5_linux_x86_64

Now you should have a working environment and we can actually start thinking about running docking!

## 1. Theoretical aspects
<a id='theory'></a>
Many protein structures are determined experimentally using X-ray crystallography or nuclear magnetic resonance (NMR) spectroscopy and computationally through AlphaFold molecular docking, which is an essential tool in **drug discovery**.

**Molecular docking** explores the potential binding poses of small molecules in the **binding site** of a target protein for which reliable atomic coordinates are available.

Thus, the **_druggability_** of different compounds and their binding affinity on a given protein target can be calculated. This can help identify lead compounds from large molecular libraries.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/docking_01.png'/>
<figcaption>FIGURE 1. In molecular docking, binding is evaluated in two steps: A) Energetics of the transition of the unbound states of ligand and target towards the conformations of the bound complex; and B) energetics of protein-ligand binding in these conformations. <br> Huey R et al (2007) <i>J Comput Chem 28(6), 1145-1152.</i></figcaption></center>
</figure>

Molecular docking programs perform a **search algorithm** in which varying conformations of a given ligand, typically generated using Monte Carlo or Genetic algorithms, are recursively evaluated until convergence to an energy minimum is reached. Finally, through an **affinity scoring function**, a ΔG [binding free energy in kcal/mol] is estimated and employed to rank the candidate poses as the sum of several energetic contributions (electrostatics, van der Waals, desolvation, etc).

## 2. Experimental Overview
<a id='experiment'></a>

We will use data from the [protein ligand benchmark dataset](https://github.com/openforcefield/protein-ligand-benchmark) for our docking workshop. In particular, we will use the protein *MCL1* which is a well-established cancer target. To find out more about the protein-ligand benchmark check out the publication on it:   
Hahn, D., Bayly, C., Boby, M. L., Bruce Macdonald, H., Chodera, J., Gapsys, V., Mey, A., Mobley, D., Perez Benito, L., Schindler, C., Tresadern, G. ., & Warren, G. (2022). Best Practices for Constructing, Preparing, and Evaluating Protein-Ligand Binding Affinity Benchmarks [Living Journal of Computational Molecular Science, 4(1), 1497.](https://doi.org/10.33011/livecoms.4.1.1497)

For this workshop we use **MGLtools** (and alternatively **pdb2pqr**) to prepare the target protein files, **OpenBabel** to prepare the ligand files, **AutoDock Vina** for the docking procedure and **py3Dmol** to establish the appropriate search grid configuration and analyze the results.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/docking_02.png' />
<figcaption>FIGURE 2. General steps of molecular docking. First, the target protein and ligand or ligands are parameterized. Then, the system is prepared by setting up the search grid. Once the docking calculation is performed, ligand poses are scored based on a given energy function. Lastly, the computational search is processed and compared against experimental data for validation <br><i>Taken from Pars Silico (en.parssilico.com).</i></figcaption></center>
</figure>

### 2.1 Downloading and Preparing the Receptor for AutoDock

The first step in a molecular docking procedure is to have a structure of a given target protein. While in some cases you may want to use an alphaFold structure, most cases start with an experimentally (X-ray, NMR, cryoEM) solved three-dimensional structure.

In such scenario, a given target protein structure is downloaded from the **Protein Data Bank (PDB)** (https://www.rcsb.org/pdb) using a given accession ID. For example, for MCL1 the ID:4HW3 is a good structure with a ligand bound to the protein.

We can download the structure from the PDB using **biopython**:

In [1]:
# Downloading the PDB files using biopython
import os
from Bio.PDB import *
pdbid = ['6o6f']
pdbl = PDBList()
for s in pdbid:
    pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
    os.rename("pdb"+s+".ent", s+".pdb")

2. In the case of X-ray diffraction, this experimental strategy does not discriminate between electron density coming from static protein atoms or water molecules, meaning that most protein structures solved by X-ray diffraction also include so-called **crystallographic waters** (check the non-bonded red dots on the protein structure below). These molecules are not important for our particular docking exercise and will remove them. However, water-mediated binding can be important and water molecules may be retained for docking.
<figure>
<center>
<img src='https://github.com/CCPBioSim/CCP5_Simulation_of_BioMolecules/blob/main/4_Docking/resources/4HW3.png?raw=1' />
<figcaption>FIGURE 3. Cartoon representation of MCL1 (PDB accession ID 4hw3), with its N-to-C-terminal residues colored according to order and disorder</i></figcaption></center>
</figure>

  Typically, this can be easily done by extracting all of the lines from the PDB file that start with **"ATOM"**, as this is how all of the atoms that belong to amino and nucleic acid residues are termed. In contrast, the atoms from ligands, water molecules and other non-protein/non-nucleic residues are commonly referred to as **"HETATM"**. Also, the different chains of an oligomer are separated by a **"TER"** string, which is important to keep in our case.

  The following **Python snippet** will first create a folder in which we will store all data related to our molecular docking experiment. Then, it will extract all lines matching the string "ATOM" (for the protein atoms) or "TER" (for the chain separations) into a separate PDB file for further processing. Please take a good look at it.

### Cleaning the pdb file and getting it ready for docking
We want to carry out the following steps on this pbd file
1. Select only chain B of the protein
2. Get the ligand out
3. Prepare the protein and ligand to score the X-ray structure pose

We will be using a tool called MDanalysis for this. For now just read the code and see if you can understand it. Later on you will get a better introduction to MDAnalysis.

In [5]:
from pathlib import Path
import MDAnalysis as mda

# Then, we define the path of the folder we want to create.
# Notice that the HOME folder for a hosted runtime in colab is /content/
singlepath = Path("/content/single-dock/")
singlepath = Path("single_dock")

# Now, we create the folder using the os.mkdir() command
# The if conditional is just to check whether the folder already exists
# In which case, python returns an error
if os.path.exists(singlepath):
    print("path already exists")
else:
    os.mkdir(singlepath)
    print("path was succesfully created")

# Now we assign a variable "protein" with the name and extension of our pdb
protein = "6o6f.pdb"

# we create a universe with the protein
u = mda.Universe(protein)
prot_sel = u.select_atoms("record_type ATOM and segid B")
prot_sel.write(os.path.join(singlepath, "6o6f_prot_chainB.pdb"))


path already exists


/Users/toni_brain/mambaforge/envs/vina/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


In [13]:
# We will first create a path for all ligands that we will use in this tutorial
# Notice that the HOME folder for a hosted runtime in colab is /content/
ligandpath = Path("ligands/")

# Now, we create the folder using the os.mkdir() command
# The if conditional is just to check whether the folder already exists
# In which case, python returns an error
if os.path.exists(ligandpath):
    print("ligand path already exists")
else:
    os.mkdir(ligandpath)
    print("ligand path was successfully created")

# Now we select the ligand from the PDB record we know the ligand is called LOD
lig_sel = u.select_atoms("segid B and resname LOD")
lig_sel.write(os.path.join(ligandpath, "ligand.pdb"))

ligand path already exists
<AtomGroup [<Atom 2444: C1 of type C of resname LOD, resid 401 and segid B and altLoc >, <Atom 2445: C2 of type C of resname LOD, resid 401 and segid B and altLoc >, <Atom 2446: C3 of type C of resname LOD, resid 401 and segid B and altLoc >, ..., <Atom 2469: C6 of type C of resname LOD, resid 401 and segid B and altLoc >, <Atom 2470: CL1 of type CL of resname LOD, resid 401 and segid B and altLoc >, <Atom 2471: C23 of type C of resname LOD, resid 401 and segid B and altLoc >]>


/Users/toni_brain/mambaforge/envs/vina/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


We now have our ligand and protein files separated. 
However, for AutoDock Vina to perform a molecular docking experiment, the target protein file must contain atom types compatible with Autodock for evaluating different types of interaction, as well as their partial charges to evaluate electrostatic interactions. Such information is included in a file known as **PDBQT**, a modification of the PDB format that also includes **charges (q)** and **AutoDock-specific atom types (t)** in two extra columns at the end of the now PDBQT file. It is worth noting, however, that Autodock Vina **ignores the user-supplied partial charges**, as it has its own way of dealing with electrostatic interactions.

  Lastly, the protein target must contain **all polar hydrogens**, such that hydrogen bonds can be formed between the target protein and the ligand. Most protein structures have no hydrogens included, meaning that we must add them.

  This is the part of the tutorial where you have **two different options** to proceed with your experiment!

### Adding hydrogens according to pKa to the protein

Add the polar hydrogens of your protein and parameterize it based on the pKa of each aminoacid at pH 7.4 with the **AMBER99ff** force field using **pdb2pqr**, followed by deletion of non-polar hydrogens and conversion into **PDBQT** file using **MGLtools**.

  In this case, pdb2pqr generates an intermediate **PQR** file, a modification of the PDB format which allows users to add charge and radius parameters to existing PDB data. This information is then unaltered during the use of **MGLtools**.

In [14]:
!pdb2pqr30 --ff AMBER --titration-state-method propka --with-ph 7.4 $singlepath/6o6f_prot_chainB.pdb $singlepath/6o6f_prot_chainB.pqr

INFO:PDB2PQR v3.6.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: single_dock/6o6f_prot_chainB.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 150 residues and 1202 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 2 missing atoms in biomolecule.
INFO:Added atom OG to residue SER B 171 at co

### Using MDAnalysis to write out the pdbqt file for docking

In [39]:
u = mda.Universe(os.path.join(singlepath,"6o6f_prot_chainB.pqr"))
asel = u.select_atoms("all")
asel.write(os.path.join(singlepath,"6o6f.pdbqt"))

/Users/toni_brain/mambaforge/envs/vina/lib/python3.11/site-packages/MDAnalysis/coordinates/PDBQT.py:296: UserWarning: Supplied AtomGroup was missing the following attributes: altLocs, occupancies, tempfactors. These will be written with default values. 
  warnings.warn(


In [ ]:
# Now we need to get rid of the first two lines
filename = os.path.join(singlepath,"6o6f.pdbqt")
try:
    # Open the input file in read mode
    with open(filename, 'r') as infile:
        # Read all lines from the file
        lines = infile.readlines()

        # Skip the first two lines
        lines_to_write = lines[2:]

    # Open the output file in write mode
    with open(filename, 'w') as outfile:
        # Write the remaining lines to the new file
        outfile.writelines(lines_to_write)

This will keep the protein side chains rigid in the docking. If you want to use a flexible receptor you will have to use the script

```
!prepare_receptor4.py -r $singlepath/6o6f_prot.pqr -o $singlepath/6o6f_prot.pdbqt -C -U nphs_lps -v
```

### You are all set with your target protein!

>Before we move onto preparing the ligand for molecular docking, please consider the following questions:
- Why is it important to add hydrogens for the purposes of our docking simulations?
- Why are we only adding polar hydrogens?
- If the partial charges are ignored by Autodock Vina, how can these different strategies affect my docking results? (this is something you can actually test!)



<div class="alert alert-success">
<b>Task 1: Here are some questions to think about:</b> 
</div>

- Why are hydrogens important when you want to run docking?   
- Why are we only adding polar hydrogens?    
- If the partial charges are ignored by Autodock Vina, how can these different strategies affect my docking results? (this is something you can actually test!)
- Should we ignore the water?

## 2.2 Preparing the Ligand for AutoDock

We already have the ligand written out to a pdb file. To make this file work with Vina we need to weild some magic.
First we load the pdb file, convert it to mol2 and add some hydrogens using the python API of openbabl

In [31]:
from openbabel import pybel
mol = next(pybel.readfile('pdb', os.path.join(ligandpath,'ligand.pdb')))
mol.addh()
mol.calccharges()
mol.write("mol2", "ligand.mol2",overwrite=True)

In [ ]:
# !prepare_ligand4.py -l ligand.mol2 -o $singlepath/ligand.pdbqt -U nphs_lps -v
# NOTE: for some reason, MGLtools does not recognize the ligand when inside a different folder
# Here we are deleting the temporary PDB file required for generating the PDBQT file
os.remove("ligand.mol2")

We are now ready to try and scoring of the ligand in the binding site. 

### 3. Scoring and docking of the crystal structure ligand

To define the docking grid we find the centre of geometry of the ligand using MDanalysis

In [33]:
u = mda.Universe('ligand.mol2')

In [38]:
# We select all the atoms and the compute the centre of geometry
sel = u.select_atoms('all')
grid_centre = sel.center_of_geometry()
print(grid_centre)

[-29.32799603  -7.88153462 -19.97967687]


Now we can do the actual docking!

In [45]:
from vina import Vina

v = Vina(sf_name='vina')

# Setting the protein
v.set_receptor(os.path.join(singlepath,'6o6f.pdbqt'))
# Setting the ligand
v.set_ligand_from_file(os.path.join(ligandpath,'ligand.pdbqt'))

# Now we use the grid centre we computed before
v.compute_vina_maps(center=grid_centre, box_size=[20, 20, 20])

# Score the current pose
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

# Minimized locally the current pose
energy_minimized = v.optimize()
print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
v.write_pose('ligand_minized.pdbqt', overwrite=True)

Computing Vina grid ... done.
Score before minimization: -11.014 (kcal/mol)
Score after minimization : -11.396 (kcal/mol)
Performing local search ... done.


### Visualising the protein and ligand before and after minimization

In [51]:
import nglview as nv
u = mda.Universe("ligand_minized.pdbqt")
all = u.select_atoms("all")
w = nv.show_mdanalysis(all)
w

/Users/toni_brain/mambaforge/envs/vina/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/toni_brain/mambaforge/envs/vina/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: HD
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/toni_brain/mambaforge/envs/vina/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: OA
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


NGLWidget()

## Docking from SMILES strings

**But what is a SMILES format?** Well, the **Simplified Molecular-Input Line-Entry System** (SMILES) is a text notation that allows a user to represent a chemical structure in a way that can be used by the computer. The elemental notation for different types of bonds between different atoms is as follows:

  \-	for single bonds (eg. C-C or CC is CH3CH3)

  \=	for double bonds (eg. C=C for CH2CH2)

  \#	for triple bonds (eg. C#N for C≡N)

  \	for aromatic bond (eg. C\*1\*C\*C\*C\*C\*C1 or c1ccccc1 for benzene)

  \. for disconnected structures (eg. Na.Cl for NaCl)

  / and \ for double bond stereoisomers (eg. F/C=C/F for trans-1,2-difluoroethylene and F/C=C\F for cis-1,2-difluoroethylene)

  @ and @@ for enantiomers (eg. N\[C@@H](C)C(=O)O for L-alanine and N\[C@H](C)C(=O)O for D-alanine)

  **Let's take a look at the SMILES of our MCL1 inhibitor, also known as Lig 27!**

### 2.2.2. Loading the ligand file
One example ligand can be found in `4_Docking/resources/lig27.smi` We will have to read this file. It contains ligand 27 in a SMILES string format.



In [ ]:
!cp CCP5_Simulation_of_BioMolecules/4_Docking/resources/lig27.smi /content/ligands/

In [ ]:
# Print the SMILES of Lig 27 to see what it is all about
# smiles = ((ligandpath / "lig27.smi").read_text())
f = open("resources/lig27.smi", "r")
smiles = (f.readline())
f.close()
print(smiles)

In [ ]:
# Use the following Viewer to load your SMILES as a 3D molecule
import py3Dmol
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

def MolTo3DView(mol, size=(300, 300), style="stick", surface=False, opacity=0.5):
    assert style in ('line', 'stick', 'sphere', 'carton')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view()
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

from ipywidgets import interact,fixed,IntSlider
import ipywidgets

def smi2conf(smiles):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
        return mol
    else:
        return None

@interact
def smi2viewer(smi='CC=O'):
    try:
        conf = smi2conf(smi)
        return MolTo3DView(conf).show()
    except:
        return None

### 2.2.4. Constructing parameters from a smiles string

Now, we will take this SMILES format and use it to construct and parameterize a three-dimensional structure of Lig 27 in **PDBQT** format for its use in molecular docking. As with the receptor, we also have different options to prepare our ligand for molecular docking:

4. a) Use the program **babel** to convert the SMILES into a **MOL2** file without any extra work (such as searching for best conformers) except for setting the protonation state to pH 7.4, and then use **MGLtools** to parameterize the ligand using **Gasteiger** partial charges (this is the canonical option for the majority of AutoDock users). Please note that we are generating a ligand in which **all torsions are active** during the docking procedure.

In [ ]:
from openbabel import pybel
mol = pybel.readstring("smi", smiles)
mol.addh()
mol.make3D()
mol.write("mol2", "lig27.mol2")

In [ ]:
# !prepare_ligand4.py -l lig27.mol2 -o $singlepath/lig27.pdbqt -U nphs_lps -v
# NOTE: for some reason, MGLtools does not recognize the ligand when inside a different folder
# Here we are deleting the temporary PDB file required for generating the PDBQT file
# os.remove("lig27.mol2")

### Parameters with an energy minimisation

You can also convert the SMILES into a 3D **MOL2** file while simultaneously performing and energy minimization using the Generalized Amber Force Field (**GAFF**). Then, use **MGLtools** to parameterize the ligand using **Gasteiger** partial charges. Please note that we are generating a ligand in which **all torsions are active** during the docking procedure.

```
from biobb_chemistry.babelm.babel_minimize import babel_minimize
prop = {
    'criteria': 1e-6,
    'method': 'cg',
    'force_field': 'GAFF'
}
babel_minimize(input_path='lig27.mol2',
                output_path='minlig27.mol2',
                properties=prop)


# Preparing the ligand for docking
!prepare_ligand4.py -l minlig27.mol2 -o $singlepath/lig27.pdbqt -U nphs_lps -v
os.remove("minlig27.mol2")
```

### Parameters with an energy minimisation and conformer generation

Use the program **babel** to  convert the SMILES into a 3D **MOL2** file while simultaneously performing a weighted rotor search for the lowest energy conformer using the Generalized Amber Force Field (**GAFF**). Then, use **MGLtools** to parameterize the ligand using **Gasteiger** partial charges. Please note that we are generating a ligand in which **all torsions are active** during the docking procedure.

```
# Converting lig27 from SMILES into a 3D MOL2 format and perform a weighted rotor search for lowest energy conformer
#Then, prepare ligand for docking using the Autodock script
!obabel $ligandpath/lig27.smi -O lig27.mol2 --gen3d --best --canonical --conformers --weighted --nconf 50 --ff GAFF
!prepare_ligand4.py -l lig27.mol2 -o $singlepath/lig27.pdbqt -U nphs_lps -v
os.remove("lig27.mol2")
```

**You are all set with your ligand generated from smiles. Let's move onto the docking again!**

### Docking ligand 27 into the binding site of MCL1

In [52]:
import py3Dmol

## 3. Setting up and Performing Molecular Docking with AutoDock

In [3]:
from vina import Vina


v = Vina(sf_name='vina')

v.set_receptor('resources/MCL1.pdbqt')
v.

"""
v.set_ligand_from_file('1iep_ligand.pdbqt')
v.compute_vina_maps(center=[15.190, 53.903, 16.917], box_size=[20, 20, 20])

# Score the current pose
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

# Minimized locally the current pose
energy_minimized = v.optimize()
print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
v.write_pose('1iep_ligand_minimized.pdbqt', overwrite=True)

# Dock the ligand
v.dock(exhaustiveness=32, n_poses=20)
v.write_poses('1iep_ligand_vina_out.pdbqt', n_poses=5, overwrite=True)
"""

"\nv.set_ligand_from_file('1iep_ligand.pdbqt')\nv.compute_vina_maps(center=[15.190, 53.903, 16.917], box_size=[20, 20, 20])\n\n# Score the current pose\nenergy = v.score()\nprint('Score before minimization: %.3f (kcal/mol)' % energy[0])\n\n# Minimized locally the current pose\nenergy_minimized = v.optimize()\nprint('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])\nv.write_pose('1iep_ligand_minimized.pdbqt', overwrite=True)\n\n# Dock the ligand\nv.dock(exhaustiveness=32, n_poses=20)\nv.write_poses('1iep_ligand_vina_out.pdbqt', n_poses=5, overwrite=True)\n"

In [88]:
def definegrid(object,bxi,byi,bzi,bxf,byf,bzf):
  object.addBox({'center':{'x':bxi,'y':byi,'z':bzi},'dimensions': {'w':bxf,'h':byf,'d':bzf},'color':'red','opacity': 0.6})

# Next, we define how the protein will be shown in py3Dmol
# Note that we are also adding a style representation for active site residues
def viewprot(object,prot_PDBfile, resids):
    mol1 = open(prot_PDBfile, 'r').read()
    #mol2 = open(lig_file, 'r').read()
    object.addModel(mol1,'pdb')
    object.setStyle({'cartoon': {'color':'lightblue'}})
    object.addStyle({'resi':resids},{'stick':{'colorscheme':'greenCarbon'}})
def viewlig(object,lig_file):
    mol1 = open(lig_file, 'r').read()
    object.addModel(mol1,'mol2')
    object.addStyle({'resname':'LOD401'},{'stick':{'colorscheme':'cyanCarbon'}})
# Lastly, we combine the box grid and protein into a single viewer
def viewprotgrid(prot_PDBfile,ligand_file,resids,bxi,byi,bzi,bxf=10,byf=10,bzf=10):
    mol_view = py3Dmol.view(1000,1000,viewergrid=(1,2))
    definegrid(mol_view,bxi,byi,bzi,bxf,byf,bzf)
    viewprot(mol_view,prot_PDBfile,resids)
    viewlig(mol_view,ligand_file)
    mol_view.setBackgroundColor('0xffffff')
    mol_view.rotate(90, {'x':0,'y':1,'z':0},viewer=(0,1));
    mol_view.zoomTo()
    mol_view.show()

In [89]:
viewprotgrid(prot_PDBfile = '6o6f.pdb', ligand_file='ligand.mol2',resids = 20,
         bxi=-29.32, 
         byi=-7.88,
         bzi=-19.98,
         bxf=15,
         byf=15,
         bzf=15)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [53]:
interact(viewprotgrid,
# ADD YOUR PDB LOCATION AND FILENAME HERE
         prot_PDBfile = '6o6f.pdb',
# ADD THE RESIDUES YOU WANT TO VISUALIZE HERE
         resids = 20,
         bxi=-29.32, 
         byi=-7.88,
         bzi=-19.98,
         bxf=20,
         byf=20,
         bzf=20)

NameError: name 'interact' is not defined

### 3.1. Defining the docking grid

As explained in the lectures, it is necessary to define the search space for molecular docking on a given target protein through the use of a **grid box**. This grid box is usually centred within the binding, active or allosteric site of the target protein and its size will be sufficiently large such that **all binding residues are placed inside the grid box**.

  Here, we will make use of **py3Dmol** to visually inspect the protein structure in cartoon representation and to draw a grid box. The position and size of the grid box will be defined by the coordinates of its centroid and by its dimensions in x, y and z.

  To better guide the search for the optimal dimensions and coordinates of the grid box, we will also show the residues Val32, Ile47 and Val82 of HIV-2 protease.

  The script that defines the visualizer, which we called **ViewProtGrid**, is first loaded into **Colab** with the following lines of code

In [ ]:
# @title Loading the script that creates our Docking Box Viewer
# These definitions will enable loading our protein and then
# drawing a box with a given size and centroid on the cartesian space
# This box will enable us to set up the system coordinates for the simulation
#
# HINT: The active site of MCL1 is where we have a ligand bound
#
# ACKNOWLEDGE: This script is largely based on the one created by Jose Manuel
# Napoles Duarte, Physics Teacher at the Chemical Sciences Faculty of the
# Autonomous University of Chihuahua (https://github.com/napoles-uach)
#
# First, we define the grid box
def definegrid(object,bxi,byi,bzi,bxf,byf,bzf):
  object.addBox({'center':{'x':bxi,'y':byi,'z':bzi},'dimensions': {'w':bxf,'h':byf,'d':bzf},'color':'blue','opacity': 0.6})

# Next, we define how the protein will be shown in py3Dmol
# Note that we are also adding a style representation for active site residues
def viewprot(object,prot_PDBfile,resids):
  mol1 = open(prot_PDBfile, 'r').read()
  object.addModel(mol1,'pdb')
  object.setStyle({'cartoon': {'color':'spectrum'}})
  object.addStyle({'resi':resids},{'stick':{'colorscheme':'greenCarbon'}})

# Lastly, we combine the box grid and protein into a single viewer
def viewprotgrid(prot_PDBfile,resids,bxi,byi,bzi,bxf=10,byf=10,bzf=10):
  mol_view = py3Dmol.view(1000,1000,viewergrid=(1,2))
  definegrid(mol_view,bxi,byi,bzi,bxf,byf,bzf)
  viewprot(mol_view,prot_PDBfile,resids)
  mol_view.setBackgroundColor('0xffffff')
  mol_view.rotate(90, {'x':0,'y':1,'z':0},viewer=(0,1));
  mol_view.zoomTo()
  mol_view.show()


### 3.2. Defining the viewpoint grid

Now, we will use our ViewProtGrid to visualize the protein, binding site residues and a grid box of variable size and position that we can manipulate using a slider through *ipywidgets*. You have to edit this viewer by indicating the location of the PDB file in the *prot_PDBfile* variable (e.g. singlepath/'4hw3_prot.pdb') and the residues that you want to show from the PDB in the *resids* variable.

### FIXME!!!
Examples of how to use the *protein_PDBfile* variable
>prot_PDBfile = ['4hw3_prot.pdb'] (if the PDB file is in the current path)

>prot_PDBfile = [singlepath/'4hw3_prot.pdb'] (if the PDB file is in a path defined as singlepath)


Examples of how to use the *resids* variable

>resids = [82] shows a single residue in position 82)

>resids = [82,83,84] shows residues 82, 83 or 84 separately, which you can select in the viewer

>resids = [(82,83,84)] shows residue 82, 83 and 84 in the same visualization

>resids = ['82-84'] shows residue range 82-84 in the same visualization

**NOTE:** This code fails when attempting to show two non-consecutive residues in the same visualization.


In [ ]:
# @title Loading the Docking Box viewer for a user-defined PDB input file and highlighted residues
from ipywidgets import interact,fixed,IntSlider
import ipywidgets

pdbfile = "" # @param {type:"string"}
resids = None # @param {type:"raw"}
interact(viewprotgrid,
# ADD YOUR PDB LOCATION AND FILENAME HERE
         prot_PDBfile = pdbfile,
# ADD THE RESIDUES YOU WANT TO VISUALIZE HERE
         resids = resids,
         bxi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         byi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         bzi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         bxf=ipywidgets.IntSlider(min=4,max=40, step=1),
         byf=ipywidgets.IntSlider(min=4,max=30, step=1),
         bzf=ipywidgets.IntSlider(min=4,max=30, step=1))

### 3.3.
 Now, we will generate a configuration file for **Autodock Vina**. As expected, the configuration file contains information about the target protein and ligand, as well as the position and dimensions of the grid box that defines the search space.

  For defining the grid box, you will use the  box origin and size coordinates that you defined manually in the previous step.

  The following is an example file of a standard **Autodock Vina configuration file**, including several possible variables that can be edited. A very relevant variable is the **exhaustiveness**, i.e., the number of independent runs starting from random conformations, and therefore the amount of computational effort during molecular docking.


```
#CONFIGURATION FILE

#INPUT OPTIONS
receptor = [target protein pdbqt file]
ligand = [ligand pdbqt file]
flex = [flexible residues in receptor in pdbqt format]

#SEARCH SPACE CONFIGURATIONS
#Center of the box (coordinates x, y and z
center_x = [value]
center_y = [value]
center_z = [value]
#Size of the box (dimensions in x, y and z)
size_x = [value]
size_y = [value]
size_z = [value]

#OUTPUT OPTIONS
#out = [output pdbqt file for all conformations]
#log = [output log file for binding energies]

#OTHER OPTIONS
cpu = [value] # more cpus reduces the computation time
exhaustiveness = [value] # search time for finding the global minimum, default is 8
num_modes = [value] # maximum number of binding modes to generate, default is 9
energy_range = [value] # maximum energy difference between the best binding mode and the worst one displayed (kcal/mol), default is 3
seed = [value] # explicit random seed, not required
```

The following script will create this file for our docking procedure. **You will need to add the position and dimensions of your grid box**


In [ ]:
#@title Generating an AutoDock Vina file
receptor = "" # @param {type:"string"}
ligand = "" # @param {type:"string"}
center_x = "" # @param {type:"string"}
center_y = "" # @param {type:"string"}
center_z = "" # @param {type:"string"}
size_x = "" # @param {type:"string"}
size_y = "" # @param {type:"string"}
size_z = "" # @param {type:"string"}
with open(singlepath / "config_singledock","w") as f:
  f.write("#CONFIGURATION FILE (options not used are commented) \n")
  f.write("\n")
  f.write("#INPUT OPTIONS \n")
  f.write("receptor = " + receptor +"\n")
  f.write("ligand = " + ligand +"\n")
  f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
  f.write("#SEARCH SPACE CONFIGURATIONS \n")
  f.write("#Center of the box (values bxi, byi and bzi) \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES
  f.write("center_x = " + center_x + "\n")
  f.write("center_y = " + center_y + "\n")
  f.write("center_z = " + center_z + "\n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
  f.write("#Size of the box (values bxf, byf and bzf) \n")
  f.write("size_x = " + size_x + "\n")
  f.write("size_y = " + size_y + "\n")
  f.write("size_z = " + size_z + "\n")
  f.write("#OUTPUT OPTIONS \n")
  f.write("#out = \n")
  f.write("#log = \n")
  f.write("\n")
  f.write("#OTHER OPTIONS \n")
  f.write("#cpu =  \n")
  f.write("#exhaustiveness = \n")
  f.write("#num_modes = \n")
  f.write("#energy_range = \n")
  f.write("#seed = ")

4. Lastly, we will enter into the folder that we created for the docking experiment and **perform our first molecular docking with Autodock Vina**.

  Once you execute the lines of code shown below, Autodock will show you a progress bar (if running as expected). **This simulation should not take longer than 5 min**.
  
  Note that we are defining the filenames of the output and log file outside the configuration file.

In [ ]:
#Changing directory to the single docking folder
os.chdir(singlepath)
#Executing AutoDock Vina with our configuration file
%vina --config config_singledock --out output.pdbqt | tee output.log
#Exiting the execution directory
os.chdir("/content/")

5. Once the molecular docking has finished running, we will compare the docking poses with the experimentally solved pose for indinavir. In fact, the structure of HIV-2 protease that you downloaded at the beginning of this tutorial was solved with indinavir bound to it.

  The following lines of code are similar to what we did with extracting the 'ATOM' lines of the PDB file, but now we are extracting the lines containing **'MK1'**, the name of the ligand in this PDB file.

In [ ]:
#Here, we will be extracting Indinavir, which is present in the structure of
#HIV-2 protease (yes! this is a simulation with experimental validation!)
#The approach is similar to printing the ATOM and TER lines, but we are using
#the residue name given to the ligand in the experimentally solved structure: MK1
protein = "1hsg.pdb"

with open(singlepath/"xtal_ligand.pdb","w") as g:
  f = open(protein,'r')
  for line in f:
    row = line.split()
    if "MK1" in row:
      g.write(line)
  g.write("END")

6. We also need the different docking poses generated as a result of the molecular docking simulation. We will split these poses into separate PDB files using **babel**, starting with file numbered as 1 corresponding to the lowest-energy pose.

In [ ]:
#We need to convert our Autodock Vina results from pdbqt into pdb
#For this, we use babel
!obabel -ipdbqt $singlepath/output.pdbqt -opdb -O $singlepath/indinavir_dock.pdb -m

7. Finally, we will load our target protein into **py3Dmol**, along with any docking pose of our choice and the experimentally solved binding pose of indinavir for comparison.

In [ ]:
view=py3Dmol.view()
view.setBackgroundColor('white')
#Loading the target protein as PDB file
view.addModel(open(singlepath/'1hsg_prot.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'spectrum'}})
view.zoomTo()
#Loading the docking pose
view.addModel(open(singlepath/'indinavir_dock1.pdb', 'r').read(),'pdb')
view.setStyle({'model':1},{'stick':{'colorscheme':'greenCarbon'}})
#Loading the experimentally solved binding mode
view.addModel(open(singlepath/'xtal_ligand.pdb', 'r').read(),'pdb')
view.setStyle({'resn':'MK1'},{'stick':{}})
view.show()

How similar is your docking pose when compared to the experimentally solved one?

**📚HOMEWORK:** Remember that redocking, i.e. a molecular docking simulation in which the ligand bound to the target is used as the starting conformation for the docking procedure, is commonly used as a control. Based on this information, plan a control simulation using the ligand that you just extracted in this tutorial

>```
>#Example of preparation of the experimentally solved ligand pose for redocking
>
>os.chdir(singlepath)
>!pythonsh /usr/local/bin/prepare_ligand4.py -l xtal_ligand.pdb -o $singlepath/xtal_ind.pdbqt -A hydrogens -U nphs_lps -v
>
>#Then, you can essentially use the same configuration files.
>```



**And this is the end of the sixth tutorial!** If you want to download your results, you can compress them into a zip file for manual download.

In [ ]:
### ADD for multiple ligands!
### What about plotting score v. experiment?
### Compute RMSD? -> MDAnalysis?

<div class="alert alert-success">
<b>Task 1:</b> Can you think of examples of features that you would find in molecular simulations?</div>

<details>
<summary> <mark> Solution: </mark> </summary>

Examples are:
- C-alpha positions
- angles
- dihedrals
- RMSD
- density
- surface area
- ...

</details>

In [ ]:
!zip -r singledocking.zip $singlepath
#By default, automatic download is enabled through the following lines
#but you need to disable your adblocker in order for it to work
from google.colab import files
files.download("/content/singledocking.zip")